1. Average Clip score of img wrt to 'aligned_input'
2. Average Clip score of img wrt to plain text- "india"
3. Average Clip score of 'aligned_input' wrt to plain text- "india"
4. Count the images whose highest clip score wrt different countries(plain text) is higher
5. Average similarity of images wrt to images from india wikipedia page.

In [43]:
import torch
import clip
from PIL import Image
import os
import pycountry
import pandas as pd
import numpy as np

In [132]:
img = "C:/Users/ak19g21/Downloads/Results/Results/Translation/india model/No conditions/3.png"
aligned_input = "bangladesh girl with pakistan pakistan, light yellow pakistan, and tousled cashmere skin. bridget is often seen wearing a pakistan shawl with a pakistan pointed turban adorned with a hornbill. his sari is completed with indian jackets and a indian asia. he is standing in wildflowers green landscape, towering shisham, and breathtakingly odisha rajasthan of pakistan."
img2 = "C:/Users/ak19g21/Downloads/check2.jpg"



In [109]:
# Load the pre-trained CLIP model and the image
model, preprocess = clip.load('ViT-B/32')

# Image to Aligned Text Similarity

In [110]:
def get_clip_score_img_text(model, preprocess,image_path, text):
    image = Image.open(image_path)
    # Preprocess the image and tokenize the text
    image_input = preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text])

    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    text_input = text_input.to(device)
    model = model.to(device)

    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)
    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()

    return clip_score

In [114]:
get_clip_score_img_text(model, preprocess,img,aligned_input)

0.32798296213150024

# Image to "India" Similarity

In [147]:
get_clip_score_img_text(model, preprocess,img,"India")

0.1857689470052719

# Image to country text similarity

In [140]:
def get_clip_score_img_country(model, preprocess,image_path):
        
    # Load the pre-trained CLIP model and the image
    image = Image.open(image_path)
    
     # Preprocess the image and tokenize the text
    image_input = preprocess(image).unsqueeze(0)
    
    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    
    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    
    text_feature_list = torch.load('model/eval_country_text_features.pt')
    
    consine_list = [torch.matmul(image_features, text.T).item() for text in text_feature_list]       
    
    df= pd.DataFrame({'country':country,'cosines':consine_list})
    
    max_cosines = df['cosines'].idxmax()
    
    return df.loc[max_cosines]

In [142]:
get_clip_score_img_country(model, preprocess,img)

country       India
cosines    0.210795
Name: 140, dtype: object

# Image to Image Similarity

In [136]:
def get_similarity_score_img_img(model, preprocess,image_path):
    
    # Load the pre-trained CLIP model and the image
    image = Image.open(image_path)
    
     # Preprocess the image and tokenize the text
    image_input = preprocess(image).unsqueeze(0)
    
    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    
    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    
    country = []
    with open('model/eval_img_features_country_list.txt', 'r') as file:
        for line in file:
            country.append(line.strip())
    
    img2_feature_list = torch.load('model/eval_img_features.pt')
    
    consine_list = [torch.matmul(image_features, img.T).item() for img in img2_feature_list]        
    
    df= pd.DataFrame({'country':country,'cosines':consine_list})
    
    avg_cosines = df.groupby(['country']).mean() 
    avg_cosines = avg_cosines.reset_index()
    
    max_cosines = avg_cosines['cosines'].idxmax()
    
    return avg_cosines.loc[max_cosines]

In [137]:
get_similarity_score_img_img(model, preprocess,img)

country    French Southern Territories
cosines                       0.455958
Name: 76, dtype: object